In [1]:
import os, shutil
import numpy as np
import pandas as pd
from keras import models, layers, optimizers
import tensorflow as tf

from sklearn.model_selection import train_test_split
from keras import backend as K

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

#### MNIST

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images.shape, test_images.shape

((60000, 28, 28), (10000, 28, 28))

In [3]:
train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype('float32')  / 255

test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype('float32')  / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [4]:
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28*28,)))
network.add(layers.Dense(10, activation='softmax'))
network.compile(optimizer='rmsprop', 
                loss='categorical_crossentropy',
                metrics=['accuracy'])
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 7s 12ms/step - loss: 0.2590 - accuracy: 0.9243
Epoch 2/5
469/469 [==============================] - 4s 9ms/step - loss: 0.1058 - accuracy: 0.9688
Epoch 3/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0698 - accuracy: 0.9790
Epoch 4/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0505 - accuracy: 0.9846
Epoch 5/5
469/469 [==============================] - 5s 12ms/step - loss: 0.0383 - accuracy: 0.9886


In [6]:
test_loss, test_accuracy = network.evaluate(test_images, test_labels)
test_loss, test_accuracy

313/313 [==============================] - 1s 3ms/step - loss: 0.0748 - accuracy: 0.9772


(0.07482732087373734, 0.9771999716758728)

#### IMDB

In [7]:
from keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [8]:
word_index = imdb.get_word_index()
reverse_word_index = dict(
    [(value, key) for (key, value) in word_index.items()])

In [9]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [10]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [11]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])

In [12]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [13]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=8,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Epoch 1/8
30/30 [==============================] - 16s 413ms/step - loss: 0.5133 - acc: 0.7991 - val_loss: 0.3878 - val_acc: 0.8746
Epoch 2/8
30/30 [==============================] - 2s 59ms/step - loss: 0.3098 - acc: 0.9039 - val_loss: 0.3079 - val_acc: 0.8906
Epoch 3/8
30/30 [==============================] - 1s 42ms/step - loss: 0.2271 - acc: 0.9304 - val_loss: 0.3003 - val_acc: 0.8824
Epoch 4/8
30/30 [==============================] - 1s 25ms/step - loss: 0.1794 - acc: 0.9426 - val_loss: 0.2835 - val_acc: 0.8873
Epoch 5/8
30/30 [==============================] - 1s 35ms/step - loss: 0.1459 - acc: 0.9555 - val_loss: 0.2822 - val_acc: 0.8877
Epoch 6/8
30/30 [==============================] - 1s 29ms/step - loss: 0.1196 - acc: 0.9637 - val_loss: 0.2989 - val_acc: 0.8835
Epoch 7/8
30/30 [==============================] - 1s 25ms/step - loss: 0.0999 - acc: 0.9700 - val_loss: 0.3361 - val_acc: 0.8738
Epoch 8/8
30/30 [==============================] - 1s 18ms/step - loss: 0.0810 - acc: 0.

In [14]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

In [15]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=loss_values, 
                         line=dict(color='royalblue', width=6),
                         name = 'Loss'))
fig.add_trace(go.Scatter(y=val_loss_values, 
                         line=dict(color='red', width=3),
                         name = 'Validation Loss'))
fig.show()

#### Reuters

In [16]:
from keras.datasets import reuters
(train_data, train_labels), (test_data, test_labels) = \
    reuters.load_data(num_words=10000)

In [17]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results
    
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [18]:
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results
one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

In [19]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=5,
                    batch_size=20,
                    validation_data=(x_val, y_val))

Epoch 1/5
400/400 [==============================] - 14s 29ms/step - loss: 1.3866 - accuracy: 0.6958 - val_loss: 1.0364 - val_accuracy: 0.7790
Epoch 2/5
400/400 [==============================] - 6s 15ms/step - loss: 0.7485 - accuracy: 0.8327 - val_loss: 0.9272 - val_accuracy: 0.7940
Epoch 3/5
400/400 [==============================] - 8s 20ms/step - loss: 0.4830 - accuracy: 0.8946 - val_loss: 0.9012 - val_accuracy: 0.8240
Epoch 4/5
400/400 [==============================] - 10s 25ms/step - loss: 0.3455 - accuracy: 0.9243 - val_loss: 0.9638 - val_accuracy: 0.8140
Epoch 5/5
400/400 [==============================] - 5s 12ms/step - loss: 0.2717 - accuracy: 0.9407 - val_loss: 1.1354 - val_accuracy: 0.8140


In [20]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

fig = go.Figure()
fig.add_trace(go.Scatter(y=loss_values, 
                         line=dict(color='royalblue', width=6),
                         name = 'Loss'))
fig.add_trace(go.Scatter(y=val_loss_values, 
                         line=dict(color='red', width=3),
                         name = 'Validation Loss'))
fig.show()

In [21]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(y=acc, 
                         line=dict(color='royalblue', width=6),
                         name = 'Accuracy'))
fig.add_trace(go.Scatter(y=val_acc, 
                         line=dict(color='red', width=3),
                         name = 'Validation Accuracy'))
fig.show()

In [22]:
model.evaluate(x_test, one_hot_test_labels)

71/71 [==============================] - 2s 24ms/step - loss: 1.2978 - accuracy: 0.7925


[1.2978484630584717, 0.7925200462341309]